## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
!pip install pymorphy2
!pip install pendulum 
!gdown 1-LGwv9U0HTWKZu01uWgBXavVol6wXZDX

import pandas as pd
import nltk
import pendulum

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import * 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

nltk.download("punkt")
nltk.download("stopwords")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1-LGwv9U0HTWKZu01uWgBXavVol6wXZDX
To: /content/sms_spam.csv
100% 481k/481k [00:00<00:00, 104MB/s]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_csv("/content/sms_spam.csv", sep=",")
data

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


In [ ]:
data["text"] = data.apply(lambda row: word_tokenize(row["text"]), axis=1) # токенизация
data["text"] = data.apply(lambda row: [word.lower() for word in row["text"] if word.isalpha()], axis=1) # преобразование всех букв к нижнему регистру
stopwords_english = stopwords.words("english")
data["text"] = data.apply(lambda row: [word for word in row["text"] if word not in stopwords_english], axis=1) # удаление стоп-слов
analyzer = MorphAnalyzer()
data["text"] = data.apply(lambda row: " ".join([analyzer.parse(word)[0].normal_form for word in row["text"]]), axis=1) # лемматизация
data

,type,text
0,ham,hope good week checking
1,ham,k give back thanks
2,ham,also cbe pay
3,spam,complimentary star ibiza holiday cash needs ur...
4,spam,okmail dear dave final notice collect tenerife...
...,...,...
5554,ham,great role model giving much really wish day m...
5555,ham,awesome remember last time got somebody high f...
5556,spam,prize go another customer c polo ltd suite lon...
5557,spam,sms ac jsco energy high u may know ur leadersh...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["type"], test_size=0.33, random_state=42)

### Мешок n-грамм

In [ ]:
accuracy = []

for i in range(1, 9):
  vectorizer = CountVectorizer(ngram_range=(1, i))
  vect_x_train = vectorizer.fit_transform(X_train)

  clf = MultinomialNB()
  clf.fit(vect_x_train, y_train)

  vect_x_test = vectorizer.transform(X_test)
  pred = clf.predict(vect_x_test)
  accuracy.append(accuracy_score(y_test, pred))

accuracy

[0.9722070844686649,
 0.9803814713896458,
 0.9798365122615804,
 0.9803814713896458,
 0.9798365122615804,
 0.9809264305177112,
 0.9809264305177112,
 0.9809264305177112]

### tf-idf

In [ ]:
pipeline = Pipeline([
    ("vect", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

param_grid = {
    "vect__max_df": (0.5, 0.7, 0.9),
    "vect__min_df": (0.01, 0.1, 0.2),
    "vect__max_features": (5000, 10000, 50000),
    "vect__ngram_range": ((1,2),(1,3),(1,4),(1,5))
}

gscv = GridSearchCV(pipeline, param_grid)

gscv.fit(X_train, y_train)
gscv.best_score_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.8/dist-packa

0.9479039474633758

In [ ]:
gscv.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.5, max_features=5000, min_df=0.01,
                                 ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

### Символьные n-граммы

In [ ]:
accuracy = []

for i in range(3, 13):
  vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, i))
  vect_x_train = vectorizer.fit_transform(X_train)

  clf = MultinomialNB()
  clf.fit(vect_x_train, y_train)

  vect_x_test = vectorizer.transform(X_test)
  pred = clf.predict(vect_x_test)
  
  accuracy.append(accuracy_score(y_test, pred))

accuracy

[0.9416893732970028,
 0.9471389645776567,
 0.953133514986376,
 0.9596730245231607,
 0.9623978201634877,
 0.9645776566757494,
 0.9662125340599454,
 0.9667574931880109,
 0.9662125340599454,
 0.9678474114441417]

In [ ]:
vectorizer1 = CountVectorizer(ngram_range=(1,6))
vect_x_train1 = vectorizer1.fit_transform(X_train)

clf1 = MultinomialNB()
clf1.fit(vect_x_train1, y_train)

vect_x_test1 = vectorizer1.transform(X_test)
pred1 = clf1.predict(vect_x_test1)

vectorizer2 = TfidfVectorizer(max_df=0.5, max_features=5000, min_df=0.01, ngram_range=(1, 2))
vect_x_train2 = vectorizer2.fit_transform(X_train)

clf2 = MultinomialNB()
clf2.fit(vect_x_train2, y_train)

vect_x_test2 = vectorizer2.transform(X_test)
pred2 = clf2.predict(vect_x_test2)

vectorizer3 = CountVectorizer(analyzer='char', ngram_range=(3, 12))
vect_x_train3 = vectorizer3.fit_transform(X_train)

clf3 = MultinomialNB()
clf3.fit(vect_x_train3, y_train)

vect_x_test3 = vectorizer3.transform(X_test)
pred3 = clf3.predict(vect_x_test3)

In [ ]:
data = {
    "Precision" : [precision_score(y_test, pred1, pos_label="spam"), precision_score(y_test, pred2, pos_label="spam"), precision_score(y_test, pred3, pos_label="spam")],
    "Recall": [recall_score(y_test, pred1, pos_label="spam"), recall_score(y_test, pred2, pos_label="spam"), recall_score(y_test, pred3, pos_label="spam")],
    "F1-score": [f1_score(y_test, pred1, pos_label="spam"), f1_score(y_test, pred2, pos_label="spam"),f1_score(y_test, pred3, pos_label="spam")],
    "Accuracy": [accuracy_score(y_test, pred1),accuracy_score(y_test, pred2),accuracy_score(y_test, pred3)]
}
df = pd.DataFrame(data,index = ["Мешок n-грамм", "tf-idf", "Символьные n-граммы"])
df

,Precision,Recall,F1-score,Accuracy
Мешок n-грамм,0.975369,0.868421,0.918794,0.980926
tf-idf,0.879310,0.671053,0.761194,0.947684
Символьные n-граммы,0.865801,0.877193,0.871460,0.967847


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb') 
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(r'\b\w\w', 'pen pencil school university') 
print(result)

['pe', 'pe', 'sc', 'un']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
result = re.split('\.', 'I live in Saint Petersburg. I love pizza. My github is a secret information.', maxsplit=2) 
print(result)

['I live in Saint Petersburg', ' I love pizza', ' My github is a secret information.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('\d', 'DIG', 'jakbxcj123scban bajh1sbd1jashbd ajhs13bdais31da22sjd')
print(result)

jakbxcjDIGDIGDIGscban bajhDIGsbdDIGjashbd ajhsDIGDIGbdaisDIGDIGdaDIGDIGsjd


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub('\S*://\S*', '', 'https://my.itmo.ru/ asdasdasdsad https://bars.itmo.ru/bars/default asdzxcasdcas')
print(result)

 asdasdasdsad  asdzxcasdcas


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[\w-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile('@[\w\.]*')
prog.findall("abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz")

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']